# Tarea 9

In [1]:
import random
import numpy as np
import cvxpy as cp
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from sklearn.linear_model import LinearRegression

## Problem 1: On matrices

In [2]:
x = np.array([
    [1, 1.75],
    [1, 1.68],
    [1, 1.82],
    [1, 1.60],
    [1, 1.77],
    [1, 1.69],
    [1, 1.73],
    [1, 1.80],
    [1, 1.66],
    [1, 1.78],
    [1, 1.74],
    [1, 1.65],
    [1, 1.70],
    [1, 1.81],
    [1, 1.67],
    [1, 1.76],
    [1, 1.63],
    [1, 1.79],
    [1, 1.71],
    [1, 1.64]
])

y = np.array([
    72, 65, 78, 58, 75, 64, 70, 80, 62, 76,
    71, 60, 68, 79, 63, 74, 59, 77, 69, 61
])

- $ \beta = (X^TX)^{-1}Y^TX $
- $ c = X^TX $
- $ d = Y^TX $
- $ c_{inv} = (X^TX)^{-1} $
- $ beta = c_{inv}d $

### 1: Calcular $X^TX$

In [3]:
c = np.dot(x.T, x)
c

array([[20.    , 34.38  ],
       [34.38  , 59.1806]])

### 2: Obtención de $\sigma (X^TX)$

In [4]:
valores_propios, vectores_propio = np.linalg.eig(c)
valores_propios[0], valores_propios[1]

(0.020560879176876767, 79.16003912082311)

Es posible invertir la matriz $X^TX$ ya que ninguno de sus valores propios son 0.

### 3: Calcular $(X^TX)^{-1} Y^T X$

In [5]:
c_inv = np.linalg.inv(c)
beta = np.dot(c_inv, np.dot(x.T, y))
for i, b in enumerate(beta):
    print(f"B{i} = {b}")

B0 = -117.69955763084822
B1 = 108.63848611451255


### 4: What problem one has with the $X^TX$ matrix when the matrix $X$ is not of full rank in its columns, i.e., one or more of its columns is a linear combination of other columns.


In [ ]:
# pendiente hacer los cálculos a mano

## Problem 2: Simple Linear Regression

### 1: Use $\hat{\beta} = (X^TX)^{-1} Y^T X$, to find $\beta = [\beta_0,\beta_1]^T$

In [7]:
c_inv = np.linalg.inv(c)
beta = np.dot(c_inv, np.dot(x.T, y))
for i, b in enumerate(beta):
    print(f"B{i} = {b}")

B0 = -117.69955763084822
B1 = 108.63848611451255


### 2: Calculate the estimate $\hat\beta$ using CVXPY

In [8]:
# Define and solve the CVXPY problem for least squares.
beta = cp.Variable(2)
objective = cp.Minimize(cp.sum_squares(x @ beta - y))
prob = cp.Problem(objective)
prob.solve()

# Print the result.
print("\nThe optimal value is", prob.value)
print("The optimal beta is", beta.value)
print("The norm of the residual is ", cp.norm(x @ beta - y, p=2).value)


The optimal value is 22.477144261489215
The optimal beta is [-117.69955763  108.63848611]
The norm of the residual is  4.741006671740635


### 3: Calculate the estimate $\hat\beta$ using Scikit-Learn

In [9]:
x_sklearn = np.array([
    [1.75], [1.68], [1.82], [1.60], [1.77], [1.69], [1.73], [1.80], [1.66], [1.78],
    [1.74], [1.65], [1.70], [1.81], [1.67], [1.76], [1.63], [1.79], [1.71], [1.64]
])

In [10]:
X = x_sklearn
y_sklearn = y 

model = LinearRegression().fit(X, y_sklearn)
model.intercept_ ,model.coef_[0]

(-117.69955763086746, 108.63848611452441)

In [11]:
predict = model.predict(X)
predict

array([72.41779307, 64.81309904, 80.0224871 , 56.12202015, 74.59056279,
       65.8994839 , 70.24502335, 77.84971738, 62.64032932, 75.67694765,
       71.33140821, 61.55394446, 66.98586876, 78.93610224, 63.72671418,
       73.50417793, 59.38117474, 76.76333251, 68.07225362, 60.4675596 ])

## Problem 3: Multiple linear regression

In [12]:
df = pd.read_excel('Tarea 9.xlsx')
df.head()

,Ones,Height,PAL,BMR,Weight
0,1,1.75,1.6,1650,72
1,1,1.68,1.5,1500,65
2,1,1.82,1.8,1800,78
3,1,1.60,1.4,1400,58
4,1,1.77,1.7,1700,75


### 1: Direct formula

In [13]:
x_multi = df.iloc[:, :4]
y_multi = df['Weight']

In [14]:
c = np.dot(x_multi.T, x_multi)
c_inv = np.linalg.inv(c)
beta = np.dot(c_inv, np.dot(x_multi.T, y_multi))
for i, b in enumerate(beta):
    print(f"B{i} = {b}")

B0 = -57.24022393900668
B1 = 46.00789572193753
B2 = -3.9696423101995606
B3 = 0.03335466085849248


### 2: Using CVXPY

In [15]:
# Pendiente

### 3: Using Scikit-Learn

In [16]:
x_multi_sk = df.iloc[:, 1:4]
y_multi_sk = df['Weight']

In [17]:
X = x_multi_sk
y_sklearn = y_multi_sk 

model = LinearRegression().fit(X, y_sklearn)
model.intercept_ ,model.coef_[0], model.coef_[1], model.coef_[2]

(-57.24022393900688,
 46.00789572201302,
 -3.9696423101795104,
 0.03335466085831953)

In [18]:
# Add a constant to the model (intercept)
X_ols = sm.add_constant(x_multi_sk)

# Fit the OLS model
model_ols = sm.OLS(y_multi_sk, X_ols).fit()

# Print the summary of the regression
print(model_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     401.0
Date:                Sun, 10 Nov 2024   Prob (F-statistic):           2.93e-15
Time:                        10:57:00   Log-Likelihood:                -23.996
No. Observations:                  20   AIC:                             55.99
Df Residuals:                      16   BIC:                             59.97
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -57.2402     18.374     -3.115      0.0

## Problem 4_ Polynomial Regression